<a href="https://colab.research.google.com/github/ekaratnida/Data_Streaming_and_Realtime_Analytics/blob/main/Week06/pyspark/Pyspark_Structure_Streaming_kafka.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget https://mirrors.estointernet.in/apache/spark/spark-2.4.8/spark-2.4.8-bin-hadoop2.7.tgz
!tar -xvf spark-2.4.8-bin-hadoop2.7.tgz
!pip install findspark

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.8-bin-hadoop2.7"

In [3]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.ml.feature import Normalizer, StandardScaler
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.streaming import StreamingContext

In [4]:
spark = SparkSession.builder.appName("readFromCSV").master("local").getOrCreate()

In [5]:
schema1 = StructType([StructField('id', IntegerType(), True),
                      StructField('name', StringType(), True),
                      StructField('age', IntegerType(), True),
                      StructField('profession', StringType(), True),
                      StructField('city', StringType(), True),
                      StructField('salary', DoubleType(), True)])

In [8]:
%mkdir /content/data

In [9]:
customer = spark.readStream \
  .format("csv") \
  .schema(schema1) \
  .option("header",True) \
  .load(r"/content/data")

In [10]:
customer.isStreaming

True

In [11]:
customer.printSchema()

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- profession: string (nullable = true)
 |-- city: string (nullable = true)
 |-- salary: double (nullable = true)



In [25]:
avg_salary = customer \
      .groupBy("profession") \
      .agg((avg("salary").alias("average_salary")) , (count("profession").alias("count"))) \
      #.sort(desc("average_salary"))


In [26]:
query1 = avg_salary.writeStream \
  .outputMode("update") \
  .format("console") \
  .start().awaitTermination()

KeyboardInterrupt: ignored

In [ ]:
results_df = customer.select("*")

In [ ]:
query = (results_df.writeStream
  .format("json")
  .queryName("selectTable")
  .option("checkpointLocation", "/content/checkpoint")
  .option("path", "/content/results")
  .outputMode("append")
  .start()
  .awaitTermination() 
)

KeyboardInterrupt: ignored